In [19]:
# Importing the libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
#% matplotlib inline
from datetime import datetime

import plotly
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.figure_factory as ff
import plotly.express as px
from plotly import tools
from plotly.subplots import make_subplots
from plotly.offline import iplot

from textblob import TextBlob, Word, Blobber
from textblob.classifiers import NaiveBayesClassifier
from textblob.taggers import NLTKTagger

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
nltk.download('maxent_ne_chunker')
nltk.download('words')

import string
import gensim
import collections
from gensim.corpora import Dictionary

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel

from autocorrect import Speller

from bs4 import BeautifulSoup
import requests
import csv
import time

import re
import string
import gc
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/aditidutta/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/aditidutta/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/aditidutta/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [20]:
# gathering up all the data from the files received in via Google Scholar search 

df1 = pd.read_csv('data/2012.csv')
df2 = pd.read_csv('data/2013.csv')
df3 = pd.read_csv('data/2014.csv')
df4 = pd.read_csv('data/2015.csv')
df5 = pd.read_csv('data/2016.csv')
df6 = pd.read_csv('data/2017.csv')
df7 = pd.read_csv('data/2018.csv')
df8 = pd.read_csv('data/2019.csv')
df9 = pd.read_csv('data/2020.csv')
df10 = pd.read_csv('data/2021.csv')
df11 = pd.read_csv('data/2022.csv')


df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11], ignore_index=True)
df.head()

,Cites,Authors,Title,Year,Source,Publisher,ArticleURL,CitesURL,GSRank,QueryDate,...,StartPage,EndPage,ECC,CitesPerYear,CitesPerAuthor,AuthorCount,Age,Abstract,FullTextURL,RelatedURL
0,1,M Turno,'Dai che mi vesto da puttana!': Cultural Repre...,2012.0,NaN,Citeseer,https://citeseerx.ist.psu.edu/viewdoc/download...,https://scholar.google.com/scholar?cites=68233...,93,2022-07-17 10:02:25,...,NaN,NaN,1,0.1,1,1,10.0,"Nadia frantically tries to escape, chased by a...",https://citeseerx.ist.psu.edu/viewdoc/download...,https://scholar.google.com/scholar?q=related:x...
1,1,Z Vaisman,A Cognitive-Behavioral Therapy Model for the T...,2012.0,NaN,search.proquest.com,https://search.proquest.com/openview/e9d6eb306...,https://scholar.google.com/scholar?cites=14365...,27,2022-07-17 10:02:25,...,NaN,NaN,1,0.1,1,1,10.0,… to impose such a classification in our next ...,NaN,https://scholar.google.com/scholar?q=related:P...
2,0,J Lammi,A genre analysis of Iain M. Banks' The Player ...,2012.0,NaN,erepo.uef.fi,https://erepo.uef.fi/bitstream/handle/12345678...,NaN,82,2022-07-17 10:02:25,...,NaN,NaN,0,0.0,0,1,10.0,… Sentient artificial intelligence has also be...,https://erepo.uef.fi/bitstream/handle/12345678...,https://scholar.google.com/scholar?q=related:n...
3,31,MR Davidson,A nurse's guide to women's mental health,2012.0,NaN,books.google.com,https://books.google.com/books?hl=en&lr=&id=I3...,https://scholar.google.com/scholar?cites=15456...,37,2022-07-17 10:02:25,...,NaN,NaN,31,3.1,31,1,10.0,Named a 2013 Doody's Core Title! 2012 Third Pl...,NaN,https://scholar.google.com/scholar?q=related:8...
4,3,A Wright,A review of best practices in child welfare,2012.0,NaN,books.google.com,https://books.google.com/books?hl=en&lr=&id=hg...,https://scholar.google.com/scholar?cites=13797...,48,2022-07-17 10:02:25,...,NaN,NaN,3,0.3,3,1,10.0,"… the child welfare field, as BP provide the i...",http://phoenixsinclairinquiry.ca/exhibits/exhi...,https://scholar.google.com/scholar?q=related:h...


In [21]:
# View the columns
df.columns

Index(['Cites', 'Authors', 'Title', 'Year', 'Source', 'Publisher',
       'ArticleURL', 'CitesURL', 'GSRank', 'QueryDate', 'Type', 'DOI', 'ISSN',
       'CitationURL', 'Volume', 'Issue', 'StartPage', 'EndPage', 'ECC',
       'CitesPerYear', 'CitesPerAuthor', 'AuthorCount', 'Age', 'Abstract',
       'FullTextURL', 'RelatedURL'],
      dtype='object')

In [22]:
df[['ArticleURL']].iloc[10]  #not in full-text #difficult to scrape web data

ArticleURL    https://link.springer.com/article/10.1007/s105...
Name: 10, dtype: object

In [23]:
df['Source'].value_counts()

# shows a lot of preprints, and it occupies the largest share of the lot

arXiv preprint arXiv …                                   85
api.taylorfrancis.com                                    34
researchgate.net                                         27
Journal of …                                             26
Proceedings of the …                                     24
                                                         ..
CENTRAL EUROPEAN POLITICAL SCIENCE                        1
The Kaleidoscope of Gender: Prisms, Patterns, and …       1
Busted! The Truth about the 50 Most Common Internet …     1
Psychology, Crime & Law                                   1
CHI Conference on Human Factors in …                      1
Name: Source, Length: 1581, dtype: int64

In [24]:
df['Source'].isna().count() 
# The high number of NA could be because of a large number of preprints

3892

### Years

In [25]:
df['Year'] = pd.to_datetime(df['Year'], format='%Y').dt.strftime('%Y')
df = df[~(df['Year'] < '2012')] # Seems like there were a few examples which dated before 2012

In [26]:
n_years = df['Year'].value_counts().reset_index(name='total')
n_years = n_years.sort_values(by="index", ascending=False)[["index", "total"]]
n_years

,index,total
2,2022,528
0,2021,883
1,2020,553
3,2019,462
4,2018,334
5,2017,237
6,2016,175
8,2015,108
7,2014,124
9,2013,107


In [27]:
fig = make_subplots(rows=1, cols=1, specs=[[{"type": "bar"}]])

fig.add_trace(go.Bar(
     x=n_years['index'], y=n_years['total'], showlegend=False,
    text =n_years['total'], 
     name="Years"), 
     row=1, col=1)
fig.update_traces(textposition='outside', marker_line_color='rgb(8,48,107)',  marker_line_width=1.5)
fig.update_yaxes(range=[0,1000])
fig.show()

### Publishers

In [28]:
df['Publisher'].unique()

array(['Citeseer', 'search.proquest.com', 'erepo.uef.fi',
       'books.google.com', 'liebertpub.com', 'api.taylorfrancis.com',
       'Taylor & Francis', 'Springer', 'scipedia.com',
       'taylorfrancis.com', 'thescholarship.ecu.edu', 'theses.fr', nan,
       'blancharder.wordpress.com', 'dc.etsu.edu', 'journals.sagepub.com',
       'academiccommons.columbia.edu', 'lurepository.lakeheadu.ca',
       'psycnet.apa.org', 'core.ac.uk', 'researchgate.net',
       'jbe-platform.com', 'd.lib.msu.edu', 'summit.sfu.ca',
       'scholarworks.boisestate.edu', 'kisseng.kstudy.com',
       'studenttheses.uu.nl', 'stars.library.ucf.edu',
       'dr.library.brocku.ca', 'utpjournals.press',
       'digital.lib.washington.edu', 'deepblue.lib.umich.edu',
       'academia.edu', 'existenz.us', 'openaccess.wgtn.ac.nz',
       'tel.archives-ouvertes.fr', 'torrossa.com', 'Google Patents',
       'dashboard.commons.pacificu.edu',
       'research-repository.griffith.edu.au', 'etd.ceu.edu', 'tdx.cat',
      

In [29]:
df['Publisher'] = df['Publisher'].replace(['taylorfrancis.com', 'api.taylorfrancis.com'],'Taylor & Francis')
df['Publisher'] = df['Publisher'].replace(['search.proquest.com'],'ProQuest')
df['Publisher'] = df['Publisher'].replace(['taylorfrancis.com', 'api.taylorfrancis.com'],'Taylor & Francis')
df['Publisher'] = df['Publisher'].replace(['books.google.com'],'Google Books')
df['Publisher'] = df['Publisher'].replace(['erepo.uef.fi'],'UEF eRepo')
df['Publisher'] = df['Publisher'].replace(['arxiv.org'],'arXiv')
df['Publisher'] = df['Publisher'].replace(['journals.sagepub.com'],'SAGE Publications')
df['Publisher'] = df['Publisher'].replace(['dl.acm.org'],'ACM Digital Library')
df['Publisher'] = df['Publisher'].replace(['aclanthology.org'],'ACL Anthology')
df['Publisher'] = df['Publisher'].replace(['ieeexplore.ieee.org'],'IEEE Xplore')
df['Publisher'] = df['Publisher'].replace(['papers.ssrn.com'],'SSRN eLibrary')
df['Publisher'] = df['Publisher'].replace(['mdpi.com'],'MDPI')
df['Publisher'] = df['Publisher'].replace(['researchgate.net'],'ResearchGate')
df['Publisher'] = df['Publisher'].replace(['psycnet.apa.org'],'APA Psycnet')
df['Publisher'] = df['Publisher'].replace(['library.oapen.org'],'OAPEN')
df['Publisher'] = df['Publisher'].replace(['ceur-ws.org'],'CEUR Workshop')
df['Publisher'] = df['Publisher'].replace(['academia.edu'],'Academia.edu')
df['Publisher'] = df['Publisher'].replace(['emerald.com'],'Emerald Publishing')
df['Publisher'] = df['Publisher'].replace(['academic.oup.com'],'Oxford Academic')


In [30]:
# Viewing only top 20 publishers to check their count
n_publishers = df['Publisher'].value_counts().reset_index(name='total')[:20]
n_publishers = n_publishers.sort_values(by="total", ascending=False)[["index", "total"]]
n_publishers

,index,total
0,Taylor & Francis,706
1,Google Books,483
2,ProQuest,387
3,Springer,220
4,arXiv,148
5,SAGE Publications,66
6,ACM Digital Library,62
7,Elsevier,53
8,ACL Anthology,51
9,IEEE Xplore,51


In [34]:
fig = make_subplots(rows=1, cols=1, specs=[[{"type": "bar"}]])

fig.add_trace(go.Bar(
     x=n_publishers['index'], y=n_publishers['total'], showlegend=False,
    text =n_publishers['total'],
     name="Top 20 Publishers"), 
     row=1, col=1)
fig.update_traces(textposition='outside', marker_line_color='rgb(8,48,107)',  marker_line_width=1.5)
fig.update_yaxes(range=[0,800])
fig

In [32]:
# Viewing the top 10 publishers vs the others

colors = ['#1F77B4', '#FF7F0E', '#2CA02C', 'rgb(255, 255, 51)', 'rgb(127, 60, 141)', '#8C564B', '#BCBD22', '#17BECE', '#7F7F7F', '#222A2A', 'rgb(228, 26, 28)']

top10pubs = df['Publisher'].value_counts()[:10].reset_index(name='total')

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "bar"}, {"type": "pie"}]])

fig.add_trace(go.Bar(
     x=top10pubs['index'], y=top10pubs['total'], showlegend=False,
    text =top10pubs['total'],
     name="Top 10 publications"), 
     row=1, col=1)
fig.update_yaxes(range=[0,750])
fig.update_layout(uniformtext_minsize=8)
fig.update_traces(textposition='outside', marker_color=colors, marker_line_color='rgb(8,48,107)',  marker_line_width=1.5)

list_top10pubs = list(df['Publisher'].value_counts()[:10].keys())

others_count = len(df.loc[~df['Publisher'].isin(list_top10pubs)].dropna(subset=['Publisher']))
new_row = {'index': 'Others', 'total':others_count}
top10pubs = top10pubs.append(new_row, ignore_index=True)

# pull is given as a fraction of the pie radius
fig.add_trace(go.Pie(
     values=top10pubs['total'],
     labels=top10pubs['index'].values, pull=[0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025], hole=.3, 
    marker_colors=colors,
     name="Publication groups"),
    row=1, col=2)
fig.update_layout(title_text='Top 10 publications with most works on misogyny', title_x=0.5, title_y=0.985,
                 legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="center", x=0.5))

fig.show()



In [33]:
df['Source'].unique()

array([nan, "Journal of Women's Health", 'Journal of Sexual Aggression',
       ..., 'LGBTQ+ Family: An Interdisciplinary Journal',
       'Journal of Library …', 'CHI Conference on Human Factors in …'],
      dtype=object)